In [27]:
from nltk.corpus import gutenberg
import time
from numpy import mean
import sys
from collections import defaultdict

# language datasets, filtering into lowercase text

### Average increase in compression rate: <span style="color:red;">7.509938589043936 %</span>


In [28]:
macbeth = gutenberg.raw('austen-persuasion.txt')
fileids = gutenberg.fileids()
print(fileids)
data = macbeth.replace('\n', '')
def MR(data):
    def filter(str):
        str = str.lower()
        i = [x for x in range(97)] 
        j = [x for x in range(123,256)]
        k = i + j
        for c in [chr(i) for i in k]:
            str = str.replace(c, '')
        return str
    
    def lzw_encode(data):
        # Build the dictionary.
        # lowercase!!
        data = filter(data)
        global initial_size_MR 
        initial_size_MR = len(data)
        hadict = [{} for _ in range(26)]

        p = ""
        result = []
        for i in range(26):
            hadict[i][chr(ord('a') + i)] = 0

        for c in data:
            pc = p + c
            if pc in hadict[ord(c)- ord('a')]:
                p = pc
            else:
                val = ord(c)- ord('a')
                if(p == ""): result.append("#")
                elif hadict[ord(p[-1]) - ord('a')][p] == 0 : result.append('#'+p[-1])
                else: result.append(str(hadict[ord(p[-1]) - ord('a')][p]) + p[-1])
                # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
                hadict[val][pc] = len(hadict[val])
                p = c

        if p:
            val = ord(c)- ord('a')
            result.append(str(hadict[val][p]) + p[-1])
        return "".join(result)
    return lzw_encode(data)
def lzw_encode(data):
    # Build the dictionary.
    def filter(str):
        str = str.lower()
        i = [x for x in range(97)] 
        j = [x for x in range(123,256)]
        k = i + j
        for c in [chr(i) for i in k]:
            str = str.replace(c, '')
        return str
    data = filter(data)
    global initial_size_lzw 
    initial_size_lzw = len(data)
    max_dict_size = 26
    dictionary = {chr(i): i for i in range(97,123)}
    dict_size = max_dict_size
    p = ""
    result = []

    for c in data:
        pc = p + c
        if pc in dictionary:
            p = pc
        else:
            if len(p) == 1: result.append('#'+p)
            else: result.append(str(dictionary[p]))
            # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
            dictionary[pc] = str(dict_size)
            dict_size += 1
            p = c

    if p:
        result.append(str(dictionary[p]))

    return "".join(result)
def lzw_decode(compressed):
    # Build the dictionary.
    max_dict_size = 256
    dictionary = {i: chr(i) for i in range(max_dict_size)}
    dict_size = max_dict_size

    # Decoding first value
    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError("Bad compressed k: %s" % k)
        
        result.append(entry)

        # Add w+entry[0] to the dictionary.

        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    return "".join(result)

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [29]:

# Example usage
# decompressed = lzw_decode(compressed)
# print("Decompressed:", decompressed)
for fileid in fileids:
    # if fileid == macbeth:
    data = gutenberg.raw(fileid)
    print('FileId:', fileid)
    start_time_MR = time.time()
    encoded_MR = MR(data)
    end_time_MR = time.time()
    total_time_MR = end_time_MR - start_time_MR
    final_size_MR = len(encoded_MR)
    start_time_lzw = time.time()
    encoded_lzw = lzw_encode(data)
    end_time_lzw = time.time()
    total_time_lzw = end_time_lzw - start_time_lzw
    final_size_lzw = len(encoded_lzw)
    compMR = (initial_size_MR-final_size_MR)/initial_size_MR * 100
    compLZW = (initial_size_lzw-final_size_lzw)/initial_size_lzw * 100
    CompInc = compMR - compLZW
    CompRateInc.append(CompInc)
    # print('Compression Rate MR: ', compMR)
    # print('Time Taken MR:', total_time_MR)
    # print('Compression Rate LZW: ', compLZW)
    # print('Time Taken LZW:', total_time_lzw)
    print('% increase in compression rate: ', CompInc)
    print('% increase in time taken: ', (total_time_MR-total_time_lzw)/total_time_lzw * 100)
    print('absolute increase in time taken: ', total_time_MR - total_time_lzw, 'seconds')
    print()
print('\033[91mAverage increase in compression rate', mean(CompRateInc), '%\033[0m')


FileId: austen-emma.txt
% increase in compression rate:  4.6926396298845425
% increase in time taken:  42.473088580277604
absolute increase in time taken:  0.08249115943908691 seconds

FileId: austen-persuasion.txt
% increase in compression rate:  5.41221221330951
% increase in time taken:  41.52343520163805
absolute increase in time taken:  0.04206442832946777 seconds

FileId: austen-sense.txt
% increase in compression rate:  4.531457984609498
% increase in time taken:  38.11244329228775
absolute increase in time taken:  0.05608320236206055 seconds

FileId: bible-kjv.txt
% increase in compression rate:  4.199595625510391
% increase in time taken:  24.28085780001476
absolute increase in time taken:  0.2674713134765625 seconds

FileId: blake-poems.txt
% increase in compression rate:  7.221242758968085
% increase in time taken:  37.189731609546314
absolute increase in time taken:  0.003340005874633789 seconds

FileId: bryant-stories.txt
% increase in compression rate:  6.692308944236185


# language datasets, no filtering (complete text using ASCII)

### Average increase in compression rate: <span style="color:red;">8.286985093818535 %</span>


In [30]:
macbeth = gutenberg.raw('austen-persuasion.txt')
fileids = gutenberg.fileids()
print(fileids)
data = macbeth.replace('\n', '')

def MR(data):
    def lzw_encode(data):
        # Build the dictionary.
        global initial_size_MR 
        initial_size_MR = len(data)
        hadict = [{} for _ in range(256)]

        p = ""
        result = []
        for i in range(256):
            hadict[i][chr(i)] = 0

        for c in data:
            pc = p + c
            if pc in hadict[ord(c)]:
                p = pc
            else:
                val = ord(c)
                if(p == ""): result.append("#")
                elif hadict[ord(p[-1])][p] == 0 : result.append('#'+p[-1])
                else: result.append(str(hadict[ord(p[-1])][p]) + p[-1])
                # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
                hadict[val][pc] = len(hadict[val])
                p = c

        if p:
            val = ord(c)
            result.append(str(hadict[val][p]) + p[-1])
        return "".join(result)
    return lzw_encode(data)
def lzw_encode(data):
    # Build the dictionary.
    global initial_size_lzw 
    initial_size_lzw = len(data)
    max_dict_size = 256
    dictionary = {chr(i): i for i in range(256)}
    dict_size = max_dict_size
    p = ""
    result = []

    for c in data:
        pc = p + c
        if pc in dictionary:
            p = pc
        else:
            if len(p) == 1: result.append('#'+p)
            else: result.append(str(dictionary[p]))
            # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
            dictionary[pc] = str(dict_size)
            dict_size += 1
            p = c

    if p:
        result.append(str(dictionary[p]))

    return "".join(result)
def lzw_decode(compressed):
    # Build the dictionary.
    max_dict_size = 256
    dictionary = {i: chr(i) for i in range(max_dict_size)}
    dict_size = max_dict_size

    # Decoding first value
    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError("Bad compressed k: %s" % k)
        
        result.append(entry)

        # Add w+entry[0] to the dictionary.

        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    return "".join(result)

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [31]:
# Example usage
# decompressed = lzw_decode(compressed)
# print("Decompressed:", decompressed)
CompRateInc = []
for fileid in fileids:
    # if fileid == macbeth:
    data = gutenberg.raw(fileid)
    print('FileId:', fileid)
    start_time_MR = time.time()
    encoded_MR = MR(data)
    end_time_MR = time.time()
    total_time_MR = end_time_MR - start_time_MR
    final_size_MR = len(encoded_MR)
    start_time_lzw = time.time()
    encoded_lzw = lzw_encode(data)
    end_time_lzw = time.time()
    total_time_lzw = end_time_lzw - start_time_lzw
    final_size_lzw = len(encoded_lzw)
    compMR = (initial_size_MR-final_size_MR)/initial_size_MR * 100
    compLZW = (initial_size_lzw-final_size_lzw)/initial_size_lzw * 100
    CompInc = compMR - compLZW
    CompRateInc.append(CompInc)
    # print('Compression Rate MR: ', compMR)
    # print('Time Taken MR:', total_time_MR)
    # print('Compression Rate LZW: ', compLZW)
    # print('Time Taken LZW:', total_time_lzw)
    print('% increase in compression rate: ', CompInc)
    print('% increase in time taken: ', (total_time_MR-total_time_lzw)/total_time_lzw * 100)
    print('absolute increase in time taken: ', total_time_MR - total_time_lzw, 'seconds')
    print()
print('\033[91mAverage increase in compression rate', mean(CompRateInc), '%\033[0m')


FileId: austen-emma.txt
% increase in compression rate:  6.134458233895595
% increase in time taken:  29.788135306496688
absolute increase in time taken:  0.05247807502746582 seconds

FileId: austen-persuasion.txt
% increase in compression rate:  6.514372968011461
% increase in time taken:  28.389766309487115
absolute increase in time taken:  0.025911331176757812 seconds

FileId: austen-sense.txt
% increase in compression rate:  5.984054013093182
% increase in time taken:  28.069633738524985
absolute increase in time taken:  0.037062883377075195 seconds

FileId: bible-kjv.txt
% increase in compression rate:  5.336875201093861
% increase in time taken:  19.201746789150455
absolute increase in time taken:  0.19786286354064941 seconds

FileId: blake-poems.txt
% increase in compression rate:  11.328073808088487
% increase in time taken:  36.27633484029194
absolute increase in time taken:  0.002927064895629883 seconds

FileId: bryant-stories.txt
% increase in compression rate:  8.8851382502

# language datasets, no filtering (complete ASCII), 8bit fixed size encodings

### Average increase in compression rate: <span style="color:red;">?? %</span>
### INCOMPLETE

In [32]:
macbeth = gutenberg.raw('austen-persuasion.txt')
fileids = gutenberg.fileids()
print(fileids)
data = macbeth.replace('\n', '')

def MR(data):
    def lzw_encode(data):
        # Build the dictionary.
        # lowercase!!
        global initial_size_MR 
        initial_size_MR = sys.getsizeof(data)
        hadict = [{} for _ in range(256)]

        p = ""
        result = []
        for i in range(256):
            hadict[i][chr(i)] = 0

        for c in data:
            pc = p + c
            if pc in hadict[ord(c)]:
                p = pc
            else:
                val = ord(c)
                if(p == ""): result.append("#")
                elif hadict[ord(p[-1])][p] == 0 : result.append('#'+p[-1])
                else: result.append(str(hadict[ord(p[-1])][p]) + p[-1])
                # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
                hadict[val][pc] = len(hadict[val])
                p = c

        if p:
            val = ord(c)
            result.append(str(hadict[val][p]) + p[-1])
        return "".join(result)
    return lzw_encode(data)
def lzw_encode(data):
    # Build the dictionary.
    global initial_size_lzw 
    initial_size_lzw = sys.getsizeof(data)
    max_dict_size = 256
    dictionary = {chr(i): i for i in range(256)}
    dict_size = max_dict_size
    p = ""
    result = []

    for c in data:
        pc = p + c
        if pc in dictionary:
            p = pc
        else:
            if len(p) == 1: result.append('#'+p)
            else: result.append(str(dictionary[p]))
            # if dict_size < 4096:  # Typically the dictionary size is limited to 4096 entries.
            dictionary[pc] = str(dict_size)
            dict_size += 1
            p = c

    if p:
        result.append(str(dictionary[p]))

    return "".join(result)
def lzw_decode(compressed):
    # Build the dictionary.
    max_dict_size = 256
    dictionary = {i: chr(i) for i in range(max_dict_size)}
    dict_size = max_dict_size

    # Decoding first value
    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError("Bad compressed k: %s" % k)
        
        result.append(entry)

        # Add w+entry[0] to the dictionary.

        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    return "".join(result)

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [33]:

# Example usage
# decompressed = lzw_decode(compressed)
# print("Decompressed:", decompressed)
CompRateInc = []
for fileid in fileids:
    # if fileid == macbeth:
    data = gutenberg.raw(fileid)
    print('FileId:', fileid)
    start_time_MR = time.time()
    encoded_MR = MR(data)
    end_time_MR = time.time()
    total_time_MR = end_time_MR - start_time_MR
    final_size_MR = sys.getsizeof(encoded_MR)
    start_time_lzw = time.time()
    encoded_lzw = lzw_encode(data)
    end_time_lzw = time.time()
    total_time_lzw = end_time_lzw - start_time_lzw
    final_size_lzw = sys.getsizeof(encoded_lzw)
    compMR = (initial_size_MR-final_size_MR)/initial_size_MR * 100
    compLZW = (initial_size_lzw-final_size_lzw)/initial_size_lzw * 100
    CompInc = compMR - compLZW
    CompRateInc.append(CompInc)
    # print('Compression Rate MR: ', compMR)
    # print('Time Taken MR:', total_time_MR)
    # print('Compression Rate LZW: ', compLZW)
    # print('Time Taken LZW:', total_time_lzw)
    print('% increase in compression rate: ', CompInc)
    print('% increase in time taken: ', (total_time_MR-total_time_lzw)/total_time_lzw * 100)
    print('absolute increase in time taken: ', total_time_MR - total_time_lzw, 'seconds')
    print()
print('\033[91mAverage increase in compression rate', mean(CompRateInc), '%\033[0m')


FileId: austen-emma.txt
% increase in compression rate:  6.134119397601227
% increase in time taken:  32.05158136507759
absolute increase in time taken:  0.05858969688415527 seconds

FileId: austen-persuasion.txt
% increase in compression rate:  6.5136884811757945
% increase in time taken:  3.365116978861055
absolute increase in time taken:  0.004266023635864258 seconds

FileId: austen-sense.txt
% increase in compression rate:  5.98361837012737
% increase in time taken:  46.89537083807779
absolute increase in time taken:  0.06529712677001953 seconds

FileId: bible-kjv.txt
% increase in compression rate:  5.336814843178569
% increase in time taken:  25.08903643181198
absolute increase in time taken:  0.25808167457580566 seconds

FileId: blake-poems.txt
% increase in compression rate:  11.313543793518663
% increase in time taken:  32.839080459770116
absolute increase in time taken:  0.0027246475219726562 seconds

FileId: bryant-stories.txt
% increase in compression rate:  8.8833931892515